## Mount Google Drive so we can Access Files

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# create a soft link from collab filesystem to Google Drive file system
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

## Clone and Build Darknet from AlexyAB

In [ ]:
%cd /content/
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
!/usr/local/cuda/bin/nvcc --version

In [ ]:
!make

## Copy in OBJ Folder with Images and Labels

In [ ]:
 # write in .zip file to copy to collab directory
!cp /mydrive/Data/obj.zip ../

In [ ]:
# write in .zip file to unzip
!unzip ../obj.zip -d data/

## Do the Config Files and Load in Names and Data Files

In [ ]:
!cp /mydrive/Replace/yolov3-obj-tiny.cfg ./cfg

In [ ]:
!cp /mydrive/Replace/obj.names ./data
!cp /mydrive/Replace/obj.data  ./data

## Generate train.txt

In [ ]:
!cp /mydrive/Scripts/generate_train.py ./

In [ ]:
!python generate_train.py

## Load in Yolo Tiny Pretrained Weights

In [ ]:
!cp /mydrive/Weights/yolov3-tiny.conv.15 ./

## Train the Model

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3-obj-tiny.cfg yolov3-tiny.conv.15 -dont_show

## Test the new Tiny Yolo

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
!cp /mydrive/Replace/yolov3-obj-tiny-test.cfg ./cfg

In [ ]:
!./darknet detector test data/obj.data cfg/yolov3-obj-tiny-test.cfg /mydrive/Backup/yolov3-obj-tiny_last.weights /mydrive/Images/jacksonhole.jpg
imShow('predictions.jpg')